In [ ]:
import pygame
import random

# Initialize Pygame
pygame.init()

# Screen dimensions
WIDTH, HEIGHT = 800, 600

# Colors
WHITE = (255, 255, 255)
BLUE = (0, 0, 255)
BLACK = (0, 0, 0)
RED = (190, 0, 0)

bullets = []

# Screen setup
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Escape the Falling Blocks")

# Clock and font
clock = pygame.time.Clock()
font = pygame.font.Font(None, 36)

# Player setup
player = pygame.Rect(WIDTH // 2, HEIGHT - 60, 50, 50)  # Player as a rectangle
player_speed = 8

# Reflection setup (the "inner enemy")
reflection = pygame.Rect(WIDTH // 2, HEIGHT - 60, 50, 50)

# Falling blocks setup
blocks = []
block_timer = 0
block_speed = 5
amount = 30
count = 0

# Game variables
score = 0
game_over = False

# Load the block image and resize it
block_image = pygame.image.load('fail.png')  # Replace with your block image file
block_width, block_height = 75, 75  # Set the desired width and height
block_image = pygame.transform.scale(block_image, (block_width, block_height))  # Resize the image

class Projectile:
    def __init__(self, x, y, radius, color):
        self.x = x
        self.y = y
        self.radius = radius
        self.color = color
        self.vel = -8  # Negative velocity to move upwards

    def draw(self, screen):
        pygame.draw.circle(screen, self.color, (self.x, self.y), self.radius)

def draw_text(text, color, x, y):
    """Draws text on the screen."""
    text_surface = font.render(text, True, color)
    screen.blit(text_surface, (x, y))

def reset_game():
    """Resets game variables for a new game."""
    global blocks, score, block_speed, game_over, reflection
    blocks = []
    score = 0
    block_speed = 5
    game_over = False
    reflection.x = player.x
    reflection.y = player.y

# Main game loop
running = True
while running:
    screen.fill(WHITE)  # Clear the screen

    # Event handling
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Get keys for movement
    keys = pygame.key.get_pressed()
    if not game_over:
        if keys[pygame.K_LEFT] and player.left > 0:
            player.x -= player_speed
        if keys[pygame.K_RIGHT] and player.right < WIDTH:
            player.x += player_speed
        if keys[pygame.K_SPACE]:
            if len(bullets) < 5:  # This will make sure we cannot exceed 5 bullets on the screen at once
                bullets.append(Projectile(round(player.x + player.width // 2), round(player.y), 6, BLACK))

    # Update the reflection (moving in opposite direction)
    if keys[pygame.K_LEFT] and reflection.left < WIDTH:
        reflection.x += player_speed  # Reflection moves opposite to player
    if keys[pygame.K_RIGHT] and reflection.right > 0:
        reflection.x -= player_speed  # Reflection moves opposite to player

    # Game over logic (collision with reflection)
    if game_over:
        draw_text(f"Game Over! Final Score: {score}", (255, 0, 0), WIDTH // 2 - 150, HEIGHT // 2 - 50)
        draw_text("Press R to Restart", BLACK, WIDTH // 2 - 100, HEIGHT // 2)
        if keys[pygame.K_r]:
            reset_game()
        pygame.display.flip()
        clock.tick(60)
        continue

    # Block generation
    block_timer += 1
    if block_timer > amount:  # Add a new block every 30 frames
        count = count + 1
        block_x = random.randint(0, WIDTH - block_width)
        blocks.append(pygame.Rect(block_x, 0, block_width, block_height))
        block_timer = 0
        if count > 10:
            amount = amount - 1
            count = 0


    # Move blocks
    for block in blocks:
        block.y += block_speed
        if block.colliderect(player):  # Check collision with player
            game_over = True
        if block.colliderect(reflection):  # Check collision with reflection (inner enemy)
            game_over = True

    # Move bullets
    for bullet in bullets:
        if 0 < bullet.y < HEIGHT:
            bullet.y += bullet.vel  # Moves the bullet upwards
        else:
            bullets.pop(bullets.index(bullet))  # This will remove the bullet if it is off the screen

    # Check for bullet-block collisions
    for bullet in bullets:
        for block in blocks:
            if block.collidepoint(bullet.x, bullet.y):
                blocks.pop(blocks.index(block))
                bullets.pop(bullets.index(bullet))
                break

    # Remove blocks that go off screen
    blocks = [block for block in blocks if block.y < HEIGHT]

    # Increase score and difficulty
    score += 1
    if score % 500 == 0:  # Every 500 points, increase speed
        block_speed += 1

    # Drawing
    pygame.draw.rect(screen, BLUE, player)  # Draw player
    pygame.draw.rect(screen, RED, reflection)  # Draw reflection (inner enemy)
    for block in blocks:
        screen.blit(block_image, block.topleft)  # Draw resized block image at the block's position
    for bullet in bullets:
        bullet.draw(screen)  # Draw bullets

    # Draw score
    draw_text(f"Score: {score}", BLACK, 10, 10)

    # Update the screen
    pygame.display.flip()
    clock.tick(60)  # 60 FPS

pygame.quit()
